### Importering av bibliotek

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Bølgelengde Kalibrering

In [8]:
#Wave_data = np.loadtxt('Data/data.txt', usecols=0, skiprows=1, delimiter=",", dtype=int)
Wave_data = np.arange(600) # Pixel positions from 0 to 599

# Define the known wavelengths at corresponding pixel positions
W1 = 400; P1= 100  # Example wavelength 1 in nm at corresponding pixel position
W2 = 800; P2= 500  # Example wavelength 2 in nm at corresponding pixel position

# Fit a linear function to the known wavelengths at the pixel positions
pixel_positions = np.array([P1, P2])
wavelengths = np.array([W1, W2])
wavelength_fit = np.polyfit(pixel_positions, wavelengths, 1)  # Linear fit
wavelength_calibration = np.poly1d(wavelength_fit)

# Calibrate the pixel positions to wavelengths
calibrated_wavelengths = wavelength_calibration(np.arange(len(Wave_data)))

df = pd.DataFrame({
    'Pixel Position': np.arange(len(calibrated_wavelengths)),
    'Calibrated Wavelength (nm)': calibrated_wavelengths
})
#print(df.to_string(index=False))
df.to_csv('Data/calibrated_wavelengths.csv', index=False)

### Intensitet Kalibrering

In [ ]:
#123abc